In [63]:
%reload_ext autoreload
%autoreload 2

In [64]:
from abc import ABC, abstractmethod

In [65]:
class Runnable(ABC):

    @abstractmethod
    def invoke(input_data):
        pass

In [66]:
import random

# multiple components ko connect karr rahe hain
class NakliLLM(Runnable):

    def __init__(self):
        print('LLM created!')

    def invoke(self,prompt):
        response_list=[
            "Delhi is capital of india",
            "IPL is cricket league",
            "AI stands for Artificial Intelligence"
        ]
        return {"response": random.choice(response_list)}
        
    def predict(self,prompt):
        response_list=[
            "Delhi is capital of india",
            "IPL is cricket league",
            "AI stands for Artificial Intelligence"
        ]
        return {"response": random.choice(response_list),"Warning":"predict() method is deprecated,use invoke() instead"}

In [67]:
llm = NakliLLM()

LLM created!


In [68]:
class NakliPromptTemplate(Runnable):

    def __init__(self,template,input_variables):
        self.template = template
        self.input_variables = input_variables

    def invoke(self,input_dict):
        return self.template.format(**input_dict)

    def format(self,input_dict):
        return [self.template.format(**input_dict),
               "Warning: format() method is deprecated,use invoke() instead"]
    

In [69]:
template = NakliPromptTemplate(
    template = 'write a {length} lines poem about {topic}',
    input_variables = ['length','topic']
)

In [70]:
class NakliStrOutputParser(Runnable):
    def __init__(self):
        pass

    def invoke(self,input_data):
        return input_data['response']

In [71]:
parser = NakliStrOutputParser()

In [90]:
# Idea for forming chains from standardized components
class RunnableConnector(Runnable):
    def __init__(self,runnable_list):
        self.runnable_list = runnable_list

    def invoke(self,input_data):
        for runnable in self.runnable_list:
            # input in runnable--> output--> input for next runnable
            input_data = runnable.invoke(input_data)

        return input_data

In [91]:
chain = RunnableConnector([template,llm,parser])

In [92]:
chain.invoke({'length':5,'topic':'Football'})

'IPL is cricket league'

In [93]:
#multiple chains chained together
template1 = NakliPromptTemplate(
    template = 'write a joke about {topic}',
    input_variables = ['topic']
)

In [94]:
template2 = NakliPromptTemplate(
    template = 'explain the joke {response}',
    input_variables = ['response']
)

In [95]:
llm = NakliLLM()

LLM created!


In [96]:
parser = NakliStrOutputParser()

In [97]:
chain1 = RunnableConnector([template1,llm])

In [98]:
chain2 = RunnableConnector([template2,llm,parser])

In [99]:
final_chain  = RunnableConnector([chain1,chain2])

In [100]:
final_chain.invoke({'topic':"cricket"})

'IPL is cricket league'